## VARS DwC conversion - larger data set (all species observed in 2001)

Resources:
- https://dwc.tdwg.org/terms/
- https://tools.gbif.org/dwca-validator/extension.do?id=dwc:Occurrence
- https://www.mbari.org/products/research-software/video-annotation-and-reference-system-vars/query-interface/advanced-user-guide/
- https://www.gbif.org/data-quality-requirements-occurrences

In [1]:
## Imports

import pandas as pd
import numpy as np

import re # for extracting logon info from text file

import jaydebeapi # for connecting to VARS db
import VARS # for connecting to VARS db

from datetime import datetime # for handling dates
import pytz # for handling time zones

import urllib.request, urllib.parse, json # for dealing with WoRMS API and output
import WoRMS # functions for querying WoRMS REST API

### Obtain data from VARS database

In [2]:
## Extract logon information from text file

# Get list of each line in file
filename = 'VARS_logon_info.txt'
f = open(filename, 'r')
lines = f.readlines()
f.close()

# Function for extracting information from lines
def get_single_quoted_text(s):
    """ 
    Takes string s and returns any text in s that is between the first set of single quotes, removing whitespace. 
    
    Example:
    s = "What if there's more ' than one' sest of single' quotes?"
    get_single_quoted_text(s) --> 's more'
    
    """
    
    extracted_text = re.search('''(?<=')\s*[^']+?\s*(?=')''', s)
    return(extracted_text.group().strip())

# Assign logon info
dr = get_single_quoted_text(lines[2])
name = get_single_quoted_text(lines[3])
pw = get_single_quoted_text(lines[4])
un = get_single_quoted_text(lines[5])
url = get_single_quoted_text(lines[6])

An explanation of the regex in get_single_quoted_text() can be found here: <br>
https://stackoverflow.com/questions/42002931/regex-extract-string-between-single-quotes-trim-whitespace?rq=1

In [10]:
## Build SQL query

sql = """
        SELECT index_recorded_timestamp,
               observation_uuid,
               concept,
               observation_group,
               observer,
               image_url,
               depth_meters,
               latitude,
               longitude,
               oxygen_ml_per_l,
               psi,
               salinity,
               temperature_celsius,
               video_uri,
               video_sequence_name,
               chief_scientist
        FROM annotations a
        WHERE NOT EXISTS (
           SELECT DISTINCT observation_uuid
           FROM annotations b
           WHERE (
             (  -- Delete last 2 years of annotations
             index_recorded_timestamp > DATEADD([year], - 2, GETDATE()) OR
             index_recorded_timestamp IS NULL OR
             index_recorded_timestamp < CAST('1970-01-02' AS datetime)
             )
           OR ( -- Delete embargoes by dive
             dive_number IN ('Ventana 50', 'Ventana 217', 'Ventana 218', 'Ventana 248')
              )
           OR (
             dive_number IN ('Tiburon 1001', 'Tiburon 1029', 'Tiburon 1030', 'Tiburon 1031', 'Tiburon 1032', 'Tiburon 1033', 'Tiburon 1034')
             )
           OR ( -- Delete embargoes by selectedConcept
             concept IN (
                 'Aegina sp. 1',
                 'Ctenophora',
                 'Cydippida 2',
                 'Cydippida',
                 'Intacta',
                 'Llyria',
                 'Lyrocteis',
                 'Lyroctenidae',
                 'Mertensia',
                 'Mertensiidae sp. A',
                 'Mystery Mollusc',
                 'Mystery Mollusc',
                 'Physonectae sp. 1',
                 'Platyctenida sp. 1',
                 'Platyctenida',
                 'Thalassocalycida sp. 1',
                 'Thalassocalycida',
                 'Thliptodon sp. A',
                 'Tjalfiella tristoma',
                 'Tjalfiella',
                 'Tjalfiellidae',
                 'Tuscarantha braueri',
                 'Tuscarantha luciae',
                 'Tuscarantha',
                 'Tuscaretta globosa',
                 'Tuscaretta',
                 'Tuscaridium cygneum',
                 'Tuscaridium',
                 'Tuscarilla campanella',
                 'Tuscarilla nationalis',
                 'Tuscarilla similis',
                 'Tuscarilla',
                 'Tuscarora',
                 'Tuscaroridae'
                 )
            )
        ) AND a.observation_uuid = b.observation_uuid
    ) AND index_recorded_timestamp >= CAST('2001-01-01' AS datetime) 
      AND index_recorded_timestamp <= CAST('2001-12-31' AS datetime)
    """

In [11]:
## Query the database

# Get connection
conn = VARS.get_db_conn(dr, url, un, pw, name)

# Submit query
data = VARS.get_data(conn, sql)

# Close connection
conn.close()

In [12]:
## Check data is there

print(data.shape)
data.head()

(210576, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,2001-05-14 23:50:03,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
1,2001-05-14 23:50:03,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
2,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
3,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
4,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock


**Note:** For some reason, this query didn't return any column names. I'll add them here...

In [15]:
## Add column names

data.rename(columns={
    0:'index_recorded_timestamp',
    1:'observation_uuid',
    2:'concept',
    3:'observation_group',
    4:'observer',
    5:'image_url',
    6:'depth_meters',
    7:'latitude',
    8:'longitude',
    9:'oxygen_ml_per_l',
    10:'psi',
    11:'salinity',
    12:'temperature_celsius',
    13:'video_uri',
    14:'video_sequence_name',
    15:'chief_scientist'
}, inplace=True)

data.head()

,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,2001-05-14 23:50:03,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
1,2001-05-14 23:50:03,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
2,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
3,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
4,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock


In [16]:
## Save

data.to_csv('VARS_2001_data.csv', index=False, na_rep='NaN')

### Read in saved data (if not pulled directly from the database)

In [20]:
## Load csv

path = ''
filename = 'VARS_2001_data.csv'
data = pd.read_csv(path+filename, dtype={'image_url': object})

print(data.shape)
data.head()

(210576, 16)


,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,2001-05-14 23:50:03,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
1,2001-05-14 23:50:03,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
2,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
3,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
4,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock


### Pre-processing

In [22]:
## Drop duplicate rows that arise from associations, which we don't care about here

data = data.drop_duplicates()
print(data.shape)
data.head()

(172102, 16)


,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,2001-05-14 23:50:03,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
1,2001-05-14 23:50:03,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
2,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
5,2001-10-03 17:15:41,C283DC75-BB9A-4E98-9E72-BC8EABC6EED9,rock,ROV,svonthun,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,881.400024,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,urn:tid:mbari.org:V2076-02,Ventana 2076,Charlie Paull
6,2001-10-03 17:15:41,EBE21573-7B5D-422B-87ED-83CB16F1D611,ledge,ROV,svonthun,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,881.400024,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,urn:tid:mbari.org:V2076-02,Ventana 2076,Charlie Paull


### Convert

In [24]:
## Start with basic event data and change headings

converted = data[['index_recorded_timestamp', 'video_sequence_name', 'observation_group', 'chief_scientist']]
converted = converted.rename(columns={
    'index_recorded_timestamp':'eventDate',
    'video_sequence_name':'eventID',
    'observation_group':'samplingProtocol',
    'chief_scientist':'recordedBy'
})
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-05-14 23:50:03,Tiburon 0324,ROV,David Clague
1,2001-05-14 23:50:03,Tiburon 0324,ROV,David Clague
2,2001-07-05 21:16:45,Ventana 2016,ROV,Rob Sherlock
5,2001-10-03 17:15:41,Ventana 2076,ROV,Charlie Paull
6,2001-10-03 17:15:41,Ventana 2076,ROV,Charlie Paull


In [25]:
## Remove whitespace from eventID

converted['eventID'] = [event.replace(' ', '_') for event in converted['eventID']]
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-05-14 23:50:03,Tiburon_0324,ROV,David Clague
1,2001-05-14 23:50:03,Tiburon_0324,ROV,David Clague
2,2001-07-05 21:16:45,Ventana_2016,ROV,Rob Sherlock
5,2001-10-03 17:15:41,Ventana_2076,ROV,Charlie Paull
6,2001-10-03 17:15:41,Ventana_2076,ROV,Charlie Paull


**Note** that this code also places an underscore between 'Doc' and 'Ricketts'. It's possible that using 'DocRicketts' could be preferable.

In [26]:
## Format eventDate

formatted = []

for dt in converted['eventDate']:
    
    # Convert string to datetime
    try:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S.%f') # some datetimes have milliseconds
    except ValueError:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
        
    # Assign UTC timezone
    utc = pytz.UTC
    dt = dt.astimezone(utc)
    
    # Put in ISO format string
    dt = dt.isoformat()
    
    # Save in list
    formatted.append(dt)

converted['eventDate'] = formatted
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-05-15T06:50:03+00:00,Tiburon_0324,ROV,David Clague
1,2001-05-15T06:50:03+00:00,Tiburon_0324,ROV,David Clague
2,2001-07-06T04:16:45+00:00,Ventana_2016,ROV,Rob Sherlock
5,2001-10-04T00:15:41+00:00,Ventana_2076,ROV,Charlie Paull
6,2001-10-04T00:15:41+00:00,Ventana_2076,ROV,Charlie Paull


In [27]:
## Add in occurrence-related columns from data, renaming as needed

converted['occurrenceID'] = data['observation_uuid']
converted['scientificName'] = data['concept']
converted['identifiedBy'] = data['observer']
converted['minimumDepthInMeters'] = data['depth_meters']
converted['maximumDepthInMeters'] = data['depth_meters']
converted['decimalLatitude'] = data['latitude']
converted['decimalLongitude'] = data['longitude']
converted['dissolvedOxygenInMLPerL'] = data['oxygen_ml_per_l']
converted['pressureInPsi'] = data['psi']
converted['salinity'] = data['salinity']
converted['temperatureInCelsius'] = data['temperature_celsius']
converted['image_url'] = data['image_url']
converted['video_uri'] = data['video_uri']
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,occurrenceID,scientificName,identifiedBy,minimumDepthInMeters,maximumDepthInMeters,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri
0,2001-05-15T06:50:03+00:00,Tiburon_0324,ROV,David Clague,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,vars,1948.599976,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09
1,2001-05-15T06:50:03+00:00,Tiburon_0324,ROV,David Clague,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,vars,1948.599976,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09
2,2001-07-06T04:16:45+00:00,Ventana_2016,ROV,Rob Sherlock,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,schlin,98.400002,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2016-05
5,2001-10-04T00:15:41+00:00,Ventana_2076,ROV,Charlie Paull,C283DC75-BB9A-4E98-9E72-BC8EABC6EED9,rock,svonthun,881.400024,881.400024,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02
6,2001-10-04T00:15:41+00:00,Ventana_2076,ROV,Charlie Paull,EBE21573-7B5D-422B-87ED-83CB16F1D611,ledge,svonthun,881.400024,881.400024,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02


In [28]:
## Add coordinateUncertaintyInMeters 

converted['coordinateUncertaintyInMeters'] = round(converted['minimumDepthInMeters']*0.03, 2)
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,occurrenceID,scientificName,identifiedBy,minimumDepthInMeters,maximumDepthInMeters,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters
0,2001-05-15T06:50:03+00:00,Tiburon_0324,ROV,David Clague,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,vars,1948.599976,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,58.46
1,2001-05-15T06:50:03+00:00,Tiburon_0324,ROV,David Clague,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,vars,1948.599976,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,58.46
2,2001-07-06T04:16:45+00:00,Ventana_2016,ROV,Rob Sherlock,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,schlin,98.400002,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2016-05,2.95
5,2001-10-04T00:15:41+00:00,Ventana_2076,ROV,Charlie Paull,C283DC75-BB9A-4E98-9E72-BC8EABC6EED9,rock,svonthun,881.400024,881.400024,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02,26.44
6,2001-10-04T00:15:41+00:00,Ventana_2076,ROV,Charlie Paull,EBE21573-7B5D-422B-87ED-83CB16F1D611,ledge,svonthun,881.400024,881.400024,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02,26.44


**Note** that the calculation for coordinateUncertaintyInMeters may be more complex once I've talked to Dave Caress. At the moment, I've just used a rule of thumb that Brian gave me that he may or may not have remembered correctly: that uncertainty goes like 3% of depth. Also, I've somewhat arbitrarily rounded to two decimal places. **It might be worth asking whether the depths, which are reported to 6 decimal places, are actually that accurate. Unlikely.**

In [37]:
## Get a list of unique species names

converted['scientificName'] = [name.lower().strip() for name in converted['scientificName']]
names = converted['scientificName'].unique()

In [31]:
%%time

## Look up names in WoRMS and save matched name, name ID and taxon ID to dicts ----- TAKES ~ 15 MINUTES TO DO THE ENTIRE NAMES LIST

name_id_dic, name_dic, id_dic = WoRMS.run_get_worms_from_scientific_name(names)

Url didn't work, check name,  manipulator
Url didn't work, check name,  rock
Url didn't work, check name,  ledge
Url didn't work, check name,  Hydromedusae
Url didn't work, check name,  Peeper
Url didn't work, check name,  equipment
Url didn't work, check name,  CO2
Url didn't work, check name,  bacterial
Url didn't work, check name,  Push
Url didn't work, check name,  cobble
Url didn't work, check name,  site
Url didn't work, check name,  basaltic
Url didn't work, check name,  depression
Url didn't work, check name,  sheet
Url didn't work, check name,  rock
Url didn't work, check name,  sediment
Url didn't work, check name,  talus
Url didn't work, check name,  physical
Url didn't work, check name,  GIS
Url didn't work, check name,  Radium
Url didn't work, check name,  TPC
Url didn't work, check name,  carbonate
Url didn't work, check name,  rock
Url didn't work, check name,  tumulus
Url didn't work, check name,  Homerpro
Url didn't work, check name,  Phyllospadix-Zostera
Url didn't wo

In [33]:
' Test '.strip().lower()

'test'

In [38]:
names

array(['manipulator', 'holothuroidea', 'forskalia', 'rock', 'ledge',
       'hydromedusae', 'lobata', 'peeper', 'apolemia', 'solmissus',
       'bathochordaeus sinker', 'equipment', 'co2 corral',
       'notobranchaea', 'bacterial mat', 'push corer', 'cobble',
       'octocorallia', 'site marker', 'histioteuthis heteropsis',
       'tiburonia granrojo', 'basaltic lava', 'depression',
       'mitrocoma cellularia', 'sheet flow', 'actiniaria', 'rock outcrop',
       'fritillaria', 'sediment', 'macrouridae', 'talus',
       'stellamedusa ventana', 'physical object', 'gis map',
       'radium sampler', 'munnopsidae', 'tpc', 'bathochordaeus mcnutti',
       'actinopterygii', 'tomopteris nisseni', 'cylinder',
       'vogtia serrata', 'coryphaenoides armatus-yaquinae complex',
       'bathylagus', 'carbonate', 'porifera', 'rock wall', 'tumulus',
       'homerpro', 'nereocystis luetkeana', 'decapoda',
       'phyllospadix-zostera detritus', 'lampocteis cruentiventer',
       'kiyohimea usagi',